In [105]:
# Importing Libraries
import os
import pandas as pd

# Handle Warning
import warnings
warnings.filterwarnings('ignore')

## Problem Statement and Business Overview
The significant rise in shipping demand has not been accompanied by a corresponding improvement in logistics companies' capabilities. Delayed deliveries pose risks in several sectors, particularly in retail e-commerce, where late shipments can disrupt the product supply chain and damage the retailer's credibility. Additionally, delays by shipping companies can lead to buyer dissatisfaction, potentially harming the retailer's reputation.

### Goal
Build a Binary Classifier for predicting the delay in logistic/product delivery for e-commerce order/supply dataset

### Objectives
* Analyze the data and determine the target feature (class) according to problem statement
* Data Cleaning
* Feature Extraction and EDA
* Model Training and Predictions

### Overview of Datasets
The dataset contains following csv files
1. df_customers - Customer details - customer_id, zip_code_prefix, city and state
2. df_order_items - order details - order_id, product_id, seller_id, price, shipping_charges
3. df_orders - order supply details - order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_timestamp, order_estimated_delivery_date
4. df_payments - payment details of orders - order_id, payment_sequencial, payment_type, payment_installments, payment_value
5. df_products - product details - product_id, product_catgory_name, product_weight_g, product_length_cm, product_height_cm, product_width_cm

In [106]:
# Importing Datasets into dataframes
names = ['Orders', 'Customers', 'Products', 'Payments',
         'OrderItems']
data = {}
base_path = os.path.join(r"C:\Users\vaibh\Documents\JobIntern\Projects\EcommerceOrdersSupply\Data")

for name in names:
    data[name] = pd.read_csv(os.path.join(base_path, f'df_{name}.csv'))

## Data Preparation and Cleaning

In [107]:
# Shapes of dataframes
for name in names:
    print(f'Shape of dataframe {name}: {data[name].shape}')

Shape of dataframe Orders: (89316, 7)
Shape of dataframe Customers: (89316, 4)
Shape of dataframe Products: (89316, 6)
Shape of dataframe Payments: (89316, 5)
Shape of dataframe OrderItems: (89316, 5)


In [108]:
# Columns of dataframes
for name in names:
    print(list(data[name].columns))

['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_timestamp', 'order_estimated_delivery_date']
['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
['order_id', 'product_id', 'seller_id', 'price', 'shipping_charges']


* We will merge the dataframes to create one consise dataframe containing all features for each order

In [109]:
# First check for duplicates
for name in names:
    print(f'Number of duplicates in {name} dataframe: {data[name].duplicated().sum()}')

Number of duplicates in Orders dataframe: 0
Number of duplicates in Customers dataframe: 0
Number of duplicates in Products dataframe: 61865
Number of duplicates in Payments dataframe: 0
Number of duplicates in OrderItems dataframe: 0


In [110]:
# Merge
data['Products'] = data['Products'].drop_duplicates()
df = data['Orders'].merge(data['OrderItems'], on='order_id', how='left')
df = df.merge(data['Customers'], on='customer_id', how='left')
df = df.merge(data['Products'], on='product_id', how='left')
df = df.merge(data['Payments'], on='order_id', how='left')
df.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,product_id,seller_id,price,...,customer_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,...,SP,toys,491.0,19.0,12.0,16.0,1,credit_card,1,259.14


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       89316 non-null  object 
 1   customer_id                    89316 non-null  object 
 2   order_status                   89316 non-null  object 
 3   order_purchase_timestamp       89316 non-null  object 
 4   order_approved_at              89307 non-null  object 
 5   order_delivered_timestamp      87427 non-null  object 
 6   order_estimated_delivery_date  89316 non-null  object 
 7   product_id                     89316 non-null  object 
 8   seller_id                      89316 non-null  object 
 9   price                          89316 non-null  float64
 10  shipping_charges               89316 non-null  float64
 11  customer_zip_code_prefix       89316 non-null  int64  
 12  customer_city                  89316 non-null 

In [112]:
# Changing datatypes of 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_timestamp', 'order_estimated_delivery_date'
df[['order_purchase_timestamp', 'order_approved_at', 'order_delivered_timestamp', 'order_estimated_delivery_date']] = df[['order_purchase_timestamp', 'order_approved_at', 'order_delivered_timestamp', 'order_estimated_delivery_date']].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       89316 non-null  object        
 1   customer_id                    89316 non-null  object        
 2   order_status                   89316 non-null  object        
 3   order_purchase_timestamp       89316 non-null  datetime64[ns]
 4   order_approved_at              89307 non-null  datetime64[ns]
 5   order_delivered_timestamp      87427 non-null  datetime64[ns]
 6   order_estimated_delivery_date  89316 non-null  datetime64[ns]
 7   product_id                     89316 non-null  object        
 8   seller_id                      89316 non-null  object        
 9   price                          89316 non-null  float64       
 10  shipping_charges               89316 non-null  float64       
 11  customer_zip_co

In [113]:
# Check kinds of order status
df['order_status'].value_counts()

order_status
delivered      87428
shipped          936
canceled         409
processing       273
invoiced         266
unavailable        2
approved           2
Name: count, dtype: int64

* For the current the analysis - we will only select the delivered orders

In [114]:
df = df[df['order_status'] == 'delivered'].drop(columns=['order_status'])

In [115]:
df.head(1)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,product_id,seller_id,price,shipping_charges,...,customer_state,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65,...,SP,toys,491.0,19.0,12.0,16.0,1,credit_card,1,259.14


In [116]:
# Check for Null values
df.isna().sum()

order_id                           0
customer_id                        0
order_purchase_timestamp           0
order_approved_at                  9
order_delivered_timestamp          6
order_estimated_delivery_date      0
product_id                         0
seller_id                          0
price                              0
shipping_charges                   0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
product_category_name            289
product_weight_g                  15
product_length_cm                 15
product_height_cm                 15
product_width_cm                  15
payment_sequential                 0
payment_type                       0
payment_installments               0
payment_value                      0
dtype: int64

In [117]:
# Drop rows containg null values from 'order_approved_at', 'order_delivered_timestamp' columns 
# As for null values from product-related columns - we will replace them with mean and mode accordingly
df = df.dropna(subset=['order_approved_at', 'order_delivered_timestamp'])
df.isna().sum()

order_id                           0
customer_id                        0
order_purchase_timestamp           0
order_approved_at                  0
order_delivered_timestamp          0
order_estimated_delivery_date      0
product_id                         0
seller_id                          0
price                              0
shipping_charges                   0
customer_zip_code_prefix           0
customer_city                      0
customer_state                     0
product_category_name            289
product_weight_g                  15
product_length_cm                 15
product_height_cm                 15
product_width_cm                  15
payment_sequential                 0
payment_type                       0
payment_installments               0
payment_value                      0
dtype: int64

In [118]:
# Check kinds of 'product_category_name'
# df['product_category_name'].mode()
mode_categories = df['product_category_name'].mode()[0]
df['product_category_name'] = df['product_category_name'].fillna(mode_categories)

for col in ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']:
    mean_value_of_mode_category = df[df['product_category_name']==mode_categories][col].mean()
    df[col] = df[col].fillna(mean_value_of_mode_category)

print(f'Total Null Value in df now - {df.isna().sum().sum()}')

Total Null Value in df now - 0


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87413 entries, 0 to 89315
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       87413 non-null  object        
 1   customer_id                    87413 non-null  object        
 2   order_purchase_timestamp       87413 non-null  datetime64[ns]
 3   order_approved_at              87413 non-null  datetime64[ns]
 4   order_delivered_timestamp      87413 non-null  datetime64[ns]
 5   order_estimated_delivery_date  87413 non-null  datetime64[ns]
 6   product_id                     87413 non-null  object        
 7   seller_id                      87413 non-null  object        
 8   price                          87413 non-null  float64       
 9   shipping_charges               87413 non-null  float64       
 10  customer_zip_code_prefix       87413 non-null  int64         
 11  customer_city       

## Data Transformation (Feature Extraction)

In [120]:
# Add class column 'is_late' 1 if order_delivered_timestamp is greater than order_estimated_delivery_date
df['is_late'] = (df['order_delivered_timestamp']>df['order_estimated_delivery_date']).astype(int)
df.head(1)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,product_id,seller_id,price,shipping_charges,...,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value,is_late
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65,...,toys,491.0,19.0,12.0,16.0,1,credit_card,1,259.14,0


In [121]:
# Check counts of class
df['is_late'].value_counts()

is_late
0    80676
1     6737
Name: count, dtype: int64

* The dataset is highly imbalanced - and this is natural for real-world datasets - later on we will use sampling techniques to transform the data

In [123]:
# Creating 'product_vol' column by using product's length, width, and height
df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']
df = df.drop(columns=['product_length_cm', 'product_height_cm', 'product_width_cm'])
df.head(1)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date,product_id,seller_id,price,shipping_charges,...,customer_city,customer_state,product_category_name,product_weight_g,payment_sequential,payment_type,payment_installments,payment_value,is_late,product_volume
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65,...,varzea paulista,SP,toys,491.0,1,credit_card,1,259.14,0,3648.0


In [124]:
df.columns

Index(['order_id', 'customer_id', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_timestamp',
       'order_estimated_delivery_date', 'product_id', 'seller_id', 'price',
       'shipping_charges', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'product_category_name', 'product_weight_g',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'is_late', 'product_volume'],
      dtype='object')

In [125]:
# df.to_csv('modified1.csv', index=False)